In [30]:
import geopandas as gpd
import psycopg2
import fiona

conn = psycopg2.connect(host="localhost",database="gds_final_project", user="postgres", password="postgres", options="-c search_path=parler,public")
cur = conn.cursor()


In [31]:
# create new table for parler data

tbl_create = '''Create table if not exists %s
    (gid           int                not null,
    timestamp      timestamp          not null);
'''%('parler_videos')

cur.execute(tbl_create)
conn.commit()

In [32]:
# create new table for washington dc shapefile

shp_create = '''Create table if not exists %s
    (gid           int                not null,
    city_name      varchar            not null,
    areamiles      double precision   not null,
    shape_area     varchar            not null);
'''%('dc_shp')

cur.execute(shp_create)
conn.commit()

add geometry column to parler data table in psql shell
```
alter table parler_videos
    add column geom
        geometry(Geometry,4326);
```

add geometry column to shapefile table in psql shell
```
alter table dc_shp
    add column geom
    geometry(Geometry,4326);
```

In [33]:
import os, os.path
import psycopg2
import fiona
from shapely.geometry import shape
import datetime

conn = psycopg2.connect(host="localhost",database="gds_final_project", user="postgres", password="postgres", options="-c search_path=parler,public")
cur = conn.cursor()

# import the parler data to the table I created
pntShp = 'us_parler.shp'
pntLyr = fiona.open(pntShp)
for featPnt in pntLyr:
    # get the attribute of the feature
    props = featPnt['properties']
    gid = props['OGC_FID']
    timestamp = props['TIMESTAMP']
    
    # get the geometry of features and uwe the wkt format
    geom = shape(featPnt['geometry']).wkt
    
    insert_statement = "INSERT INTO parler_videos (gid, timestamp, geom) VALUES('%s','%s', ST_GeometryFromText('%s', 4326))"%(gid, timestamp, geom)
    cur.execute(insert_statement)  
    
conn.commit()

In [34]:
import os, os.path
import psycopg2
import fiona
from shapely.geometry import shape

conn = psycopg2.connect(host="localhost",database="gds_final_project", user="postgres", password="postgres", options="-c search_path=parler,public")
cur = conn.cursor()

# import the washington D.C. shapefile to the table I created
Shp = 'Washington_DC_Boundary.shp'
Lyr = fiona.open(Shp)
for feat in Lyr:
    # get the attribute of the feature
    props = feat['properties']
    gid = props['OBJECTID']
    city_name = props['CITY_NAME']
    areamiles = props['AREAMILES']
    shape_area = props['Shape_Area']
    
    # get the geometry of features and uwe the wkt format
    geom = shape(feat['geometry']).wkt
    
    insert_statement = "INSERT INTO dc_shp (gid, city_name, areamiles, shape_area, geom) VALUES('%s', '%s', '%f', '%f', ST_GeometryFromText('%s', 4326))"%(gid, city_name, areamiles, shape_area, geom)
    cur.execute(insert_statement)  
    
conn.commit()

In [35]:
# Find all of the points that intersect washington DC and occurred on January 6th 2021
# Order results by Datetime
import psycopg2
import fiona

conn = psycopg2.connect(host="localhost",database="gds_final_project", user="postgres", password="postgres", options="-c search_path=parler,public")
cur = conn.cursor()

parler_table_query = '''
    select pv.gid, "timestamp", pv.geom
    from parler_videos pv 
    join dc_shp ds on st_intersects(pv.geom, ds.geom)
    where "timestamp" between '2021-01-06 00:00:00' and '2021-01-07 00:00:00'
    order by timestamp asc;
                        '''
cur.execute(parler_table_query)

rows = cur.fetchall()
for row in rows:
    print("The id and timestamp are: ", row[0], row[1])
    
conn.commit()

The id and timestamp are:  63852 2021-01-06 00:02:39
The id and timestamp are:  63853 2021-01-06 00:02:39
The id and timestamp are:  63854 2021-01-06 00:02:39
The id and timestamp are:  63858 2021-01-06 00:11:14
The id and timestamp are:  63859 2021-01-06 00:11:14
The id and timestamp are:  63860 2021-01-06 00:11:14
The id and timestamp are:  63861 2021-01-06 00:11:14
The id and timestamp are:  63862 2021-01-06 00:11:14
The id and timestamp are:  63863 2021-01-06 00:11:14
The id and timestamp are:  63864 2021-01-06 00:15:51
The id and timestamp are:  63865 2021-01-06 00:17:28
The id and timestamp are:  63866 2021-01-06 00:17:28
The id and timestamp are:  63867 2021-01-06 00:17:28
The id and timestamp are:  63868 2021-01-06 00:17:28
The id and timestamp are:  63870 2021-01-06 00:21:55
The id and timestamp are:  63879 2021-01-06 00:41:50
The id and timestamp are:  63880 2021-01-06 00:41:50
The id and timestamp are:  63883 2021-01-06 00:44:54
The id and timestamp are:  63885 2021-01-06 00

The id and timestamp are:  65536 2021-01-06 21:07:42
The id and timestamp are:  65542 2021-01-06 21:08:59
The id and timestamp are:  65544 2021-01-06 21:10:01
The id and timestamp are:  65545 2021-01-06 21:11:02
The id and timestamp are:  65546 2021-01-06 21:11:12
The id and timestamp are:  65551 2021-01-06 21:11:29
The id and timestamp are:  65552 2021-01-06 21:11:41
The id and timestamp are:  65554 2021-01-06 21:13:55
The id and timestamp are:  65558 2021-01-06 21:14:56
The id and timestamp are:  65559 2021-01-06 21:15:40
The id and timestamp are:  65560 2021-01-06 21:15:58
The id and timestamp are:  65561 2021-01-06 21:16:34
The id and timestamp are:  65562 2021-01-06 21:17:21
The id and timestamp are:  65564 2021-01-06 21:21:23
The id and timestamp are:  65566 2021-01-06 21:22:34
The id and timestamp are:  65569 2021-01-06 21:25:05
The id and timestamp are:  65573 2021-01-06 21:26:25
The id and timestamp are:  65574 2021-01-06 21:27:20
The id and timestamp are:  65577 2021-01-06 21

In [36]:
# Because of the way the parler data was recorded there are many points overlapping the same exact coordinates
# I am going to slighly randomize the points by a buffer of 10 feet for better visualization

conn = psycopg2.connect(host="localhost",database="gds_final_project", user="postgres", password="postgres", options="-c search_path=parler,public")
cur = conn.cursor()

parler_update_query = '''
    update parler_videos pv
    set geom = subquery.new_geom
    from (select st_generatepoints(geom, 1) as new_geom, gid from (select st_buffer(geom, 0.00005) as geom, gid from parler_videos) as p) as subquery
    where pv.gid = subquery.gid;
                        '''
cur.execute(parler_update_query)
conn.commit()

In [37]:
# The geometry of parler_points is multipoint but I need to convert it to point
conn = psycopg2.connect(host="localhost",database="gds_final_project", user="postgres", password="postgres", options="-c search_path=parler,public")
cur = conn.cursor()

parler_update_query = '''
    update parler_videos pv
    set (geom) = (SELECT(ST_Dump(geom)).geom as new_geom);
                        '''
cur.execute(parler_update_query)
conn.commit()

In [38]:
# This query is correct, now I will save it to a new shapefile
import psycopg2
import fiona
from shapely.geometry import Point, mapping
from shapely.wkb import loads

conn = psycopg2.connect(host="localhost",database="gds_final_project", user="postgres", password="postgres", options="-c search_path=parler,public")
cur = conn.cursor()

parler_table_query = '''
    select pv.gid, "timestamp", pv.geom
    from parler_videos pv 
    join dc_shp ds on st_intersects(pv.geom, ds.geom)
    where "timestamp" between '2021-01-06 00:00:00' and '2021-01-07 00:00:00'
    order by timestamp asc;
                        '''
cur.execute(parler_table_query)

schema = {
    'geometry': 'Point',
    'properties': {
        'gid': 'str: 20',
        'timestamp': 'str: 50'
    }
}

crs = {'init' : u'epsg:4326'}
       
with fiona.open('parler_points_final.shp', 'w', driver = "ESRI Shapefile", crs = crs, schema=schema) as output:
    rows = cur.fetchall()
    for row in rows:
        print("The gid and timestamp are: ", row[0], row[1])
        gid = row[0]
        timestamp = row[1]
        
        geom = loads(row[-1], hex = True)
        
        output.write({'properties':{'gid': gid,
                                    'timestamp': str(timestamp)
                                    },
                      'geometry': mapping(geom)
                     })
        
    conn.commit()

print ('You have exported the shapefile successfully')

The gid and timestamp are:  63852 2021-01-06 00:02:39
The gid and timestamp are:  63853 2021-01-06 00:02:39
The gid and timestamp are:  63854 2021-01-06 00:02:39
The gid and timestamp are:  63858 2021-01-06 00:11:14
The gid and timestamp are:  63859 2021-01-06 00:11:14
The gid and timestamp are:  63860 2021-01-06 00:11:14
The gid and timestamp are:  63861 2021-01-06 00:11:14
The gid and timestamp are:  63862 2021-01-06 00:11:14
The gid and timestamp are:  63863 2021-01-06 00:11:14
The gid and timestamp are:  63864 2021-01-06 00:15:51
The gid and timestamp are:  63865 2021-01-06 00:17:28
The gid and timestamp are:  63866 2021-01-06 00:17:28
The gid and timestamp are:  63867 2021-01-06 00:17:28
The gid and timestamp are:  63868 2021-01-06 00:17:28
The gid and timestamp are:  63870 2021-01-06 00:21:55
The gid and timestamp are:  63879 2021-01-06 00:41:50
The gid and timestamp are:  63880 2021-01-06 00:41:50
The gid and timestamp are:  63883 2021-01-06 00:44:54
The gid and timestamp are:  

The gid and timestamp are:  64812 2021-01-06 18:48:02
The gid and timestamp are:  64814 2021-01-06 18:48:41
The gid and timestamp are:  64815 2021-01-06 18:48:42
The gid and timestamp are:  64816 2021-01-06 18:48:59
The gid and timestamp are:  64817 2021-01-06 18:49:09
The gid and timestamp are:  64820 2021-01-06 18:49:43
The gid and timestamp are:  64821 2021-01-06 18:49:51
The gid and timestamp are:  64823 2021-01-06 18:50:41
The gid and timestamp are:  64824 2021-01-06 18:50:43
The gid and timestamp are:  64825 2021-01-06 18:50:43
The gid and timestamp are:  64826 2021-01-06 18:52:20
The gid and timestamp are:  64827 2021-01-06 18:52:20
The gid and timestamp are:  64829 2021-01-06 18:52:43
The gid and timestamp are:  64830 2021-01-06 18:52:43
The gid and timestamp are:  64831 2021-01-06 18:53:34
The gid and timestamp are:  64833 2021-01-06 18:53:53
The gid and timestamp are:  64834 2021-01-06 18:54:05
The gid and timestamp are:  64835 2021-01-06 18:54:05
The gid and timestamp are:  

The gid and timestamp are:  65592 2021-01-06 21:32:10
The gid and timestamp are:  65593 2021-01-06 21:32:50
The gid and timestamp are:  65594 2021-01-06 21:33:04
The gid and timestamp are:  65595 2021-01-06 21:33:04
The gid and timestamp are:  65596 2021-01-06 21:33:04
The gid and timestamp are:  65597 2021-01-06 21:33:04
The gid and timestamp are:  65598 2021-01-06 21:33:04
The gid and timestamp are:  65599 2021-01-06 21:33:04
The gid and timestamp are:  65600 2021-01-06 21:33:04
The gid and timestamp are:  65601 2021-01-06 21:33:07
The gid and timestamp are:  65602 2021-01-06 21:33:07
The gid and timestamp are:  65607 2021-01-06 21:34:48
The gid and timestamp are:  65608 2021-01-06 21:35:29
The gid and timestamp are:  65614 2021-01-06 21:38:45
The gid and timestamp are:  65615 2021-01-06 21:39:03
The gid and timestamp are:  65616 2021-01-06 21:39:03
The gid and timestamp are:  65617 2021-01-06 21:39:16
The gid and timestamp are:  65621 2021-01-06 21:40:51
The gid and timestamp are:  

now that I have a shapefile containing just the queried points that I want, I am going to use pandas and geopandas to extract the hour from the timestamp column so that I can easily map my points by hour.

In [20]:
import pandas as pd
import geopandas as gpd
import datetime
parler_points = gpd.read_file('parler_points_final.shp')

In [21]:
parler_points = pd.DataFrame(parler_points)

In [22]:
parler_points['datetime'] = pd.to_datetime(parler_points['timestamp'], errors='coerce')

In [23]:
print(parler_points)

        gid            timestamp                    geometry  \
0     63852  2021-01-06 00:02:39  POINT (-77.03100 38.89602)   
1     63853  2021-01-06 00:02:39  POINT (-77.03097 38.89602)   
2     63854  2021-01-06 00:02:39  POINT (-77.03095 38.89599)   
3     63858  2021-01-06 00:11:14  POINT (-77.04762 38.89622)   
4     63859  2021-01-06 00:11:14  POINT (-77.04761 38.89624)   
...     ...                  ...                         ...   
1187  65805  2021-01-06 23:23:58  POINT (-77.01027 38.88996)   
1188  65807  2021-01-06 23:34:27  POINT (-77.03213 38.89620)   
1189  65808  2021-01-06 23:34:27  POINT (-77.03211 38.89618)   
1190  65809  2021-01-06 23:34:27  POINT (-77.03213 38.89620)   
1191  65827  2021-01-06 23:50:40  POINT (-77.03638 38.89172)   

                datetime  
0    2021-01-06 00:02:39  
1    2021-01-06 00:02:39  
2    2021-01-06 00:02:39  
3    2021-01-06 00:11:14  
4    2021-01-06 00:11:14  
...                  ...  
1187 2021-01-06 23:23:58  
1188 2021-01-06

next extract hour from datetime

In [30]:
parler_points['time_hour'] = pd.DatetimeIndex(parler_points['datetime']).hour

In [31]:
print(parler_points)

        gid            timestamp                    geometry  \
0     63852  2021-01-06 00:02:39  POINT (-77.03100 38.89602)   
1     63853  2021-01-06 00:02:39  POINT (-77.03097 38.89602)   
2     63854  2021-01-06 00:02:39  POINT (-77.03095 38.89599)   
3     63858  2021-01-06 00:11:14  POINT (-77.04762 38.89622)   
4     63859  2021-01-06 00:11:14  POINT (-77.04761 38.89624)   
...     ...                  ...                         ...   
1187  65805  2021-01-06 23:23:58  POINT (-77.01027 38.88996)   
1188  65807  2021-01-06 23:34:27  POINT (-77.03213 38.89620)   
1189  65808  2021-01-06 23:34:27  POINT (-77.03211 38.89618)   
1190  65809  2021-01-06 23:34:27  POINT (-77.03213 38.89620)   
1191  65827  2021-01-06 23:50:40  POINT (-77.03638 38.89172)   

                datetime  time_hour  
0    2021-01-06 00:02:39          0  
1    2021-01-06 00:02:39          0  
2    2021-01-06 00:02:39          0  
3    2021-01-06 00:11:14          0  
4    2021-01-06 00:11:14          0  
...

convert back to geopandas dataframe

In [32]:
parler_points = gpd.GeoDataFrame(parler_points)

In [33]:
print(type(parler_points))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [36]:
print(parler_points)

        gid            timestamp                    geometry  \
0     63852  2021-01-06 00:02:39  POINT (-77.03100 38.89602)   
1     63853  2021-01-06 00:02:39  POINT (-77.03097 38.89602)   
2     63854  2021-01-06 00:02:39  POINT (-77.03095 38.89599)   
3     63858  2021-01-06 00:11:14  POINT (-77.04762 38.89622)   
4     63859  2021-01-06 00:11:14  POINT (-77.04761 38.89624)   
...     ...                  ...                         ...   
1187  65805  2021-01-06 23:23:58  POINT (-77.01027 38.88996)   
1188  65807  2021-01-06 23:34:27  POINT (-77.03213 38.89620)   
1189  65808  2021-01-06 23:34:27  POINT (-77.03211 38.89618)   
1190  65809  2021-01-06 23:34:27  POINT (-77.03213 38.89620)   
1191  65827  2021-01-06 23:50:40  POINT (-77.03638 38.89172)   

                datetime  time_hour  
0    2021-01-06 00:02:39          0  
1    2021-01-06 00:02:39          0  
2    2021-01-06 00:02:39          0  
3    2021-01-06 00:11:14          0  
4    2021-01-06 00:11:14          0  
...

In [35]:
parler_points.to_file("parler_points_with_hour.geojson", driver='GeoJSON')